# High Order Singular Value Decomposition (HOSVD)

In [123]:
import numpy as np
import tensorly as tl
from tensorly.tenalg import mode_dot
from numpy.random import randn, rand, randint
from numpy.linalg import svd, matrix_rank, norm
from scipy.io import loadmat
import time

In [41]:
hosvd_test = loadmat("hosvd_test.mat")
hosvd_denoising = loadmat("hosvd_denoising.mat")

## Problem 1

In [200]:
np.random.seed(0)
shape = (2, 3, 4)

X = randn(shape[0], shape[1], shape[2])

np.random.seed(1)
X = X + randn(shape[0], shape[1], shape[2])*1j

In [113]:
def Hermitian(X):
    return np.array(np.matrix(X).H)

def HOSVD(X, ml_rank=None):
    U_list = []
    S = X
    
    for n in range(X.ndim):
        X_n = tl.unfold(X, n)
        Un, _, _ = svd(X_n)
        
        if ml_rank:
            Un = Un[:, :ml_rank[n]]

        U_list.append(Un)

        S = mode_dot(S, Hermitian(Un), n)
        
    return S, U_list


S, U = HOSVD(X)
assert X.all() == mode_dot(mode_dot(mode_dot(S, U[0], 0), U[1], 1), U[2], 2).all()

## Problem 2

In [138]:
# np.random.seed(0)
# X_shape = (10, 8, 4)
# Y_shape = (5, 5, 5)

# X = randn(X_shape[0], X_shape[1], X_shape[2])
# Y = randn(Y_shape[0], Y_shape[1], Y_shape[2])

# np.random.seed(1)
# X = X + randn(X_shape[0], X_shape[1], X_shape[2])*1j
# Y = Y + randn(Y_shape[0], Y_shape[1], Y_shape[2])*1j

# X_ranks = ()

In [204]:
X = np.moveaxis(hosvd_denoising["tenX_noise"], 2, 0)
Y = np.moveaxis(hosvd_denoising["tenY_noise"], 2, 0)

In [205]:
import plotly.graph_objects as go

S, U = HOSVD(X)

fig = go.Figure(
    layout=go.Layout(
        title="Singular Values of X",
        template="simple_white"))

fig.add_trace(go.Scatter(x=[i for i in range(S.shape[0])], 
                         y=[norm(S[i,:,:], 'fro') for i in range(S.shape[0])], 
                         mode='lines+markers', name='I1'))

fig.add_trace(go.Scatter(x=[i for i in range(S.shape[1])], 
                         y=[norm(S[:,i,:], 'fro') for i in range(S.shape[1])], 
                         mode='lines+markers', name='I2'))

fig.add_trace(go.Scatter(x=[i for i in range(S.shape[2])], 
                         y=[norm(S[:,:,i], 'fro') for i in range(S.shape[2])], 
                         mode='lines+markers', name='I3'))


fig.update_yaxes(title='Singular value', showgrid=True)
fig.update_xaxes(title='index')

fig.show()

In [212]:
S, U = HOSVD(X, (5, 2, 3))

diff = tl.norm(X - mode_dot(mode_dot(mode_dot(S, U[0], 0), U[1], 1), U[2], 2))**2

NMSE = diff/(norm(X)**2)

round(NMSE, 5)

0.00012

In [207]:
S, U = HOSVD(Y)

fig = go.Figure(
    layout=go.Layout(
        title="Singular Values of Y",
        template="simple_white"))

fig.add_trace(go.Scatter(x=[i for i in range(S.shape[0])], 
                         y=[norm(S[i,:,:], 'fro') for i in range(S.shape[0])], 
                         mode='lines+markers', name='I1'))

fig.add_trace(go.Scatter(x=[i for i in range(S.shape[1])], 
                         y=[norm(S[:,i,:], 'fro') for i in range(S.shape[1])], 
                         mode='lines+markers', name='I2'))

fig.add_trace(go.Scatter(x=[i for i in range(S.shape[2])], 
                         y=[norm(S[:,:,i], 'fro') for i in range(S.shape[2])], 
                         mode='lines+markers', name='I3'))


fig.update_yaxes(title='Singular value', showgrid=True)
fig.update_xaxes(title='index')

fig.show()

In [213]:
S, U = HOSVD(Y, (1, 1, 1))

diff = tl.norm(Y - mode_dot(mode_dot(mode_dot(S, U[0], 0), U[1], 1), U[2], 2))**2

NMSE = diff/(norm(Y)**2)

round(NMSE, 5)

0.02047

## Higher-Order Orthogonal Iteration (HOOI)

In [12]:
def HOOI(X):    
    _, U = HOSVD(X)

    U_list = []
    for i in range(X.ndim):

        Un_tilde = X
        for j, Un in enumerate(U): 
            if j != i:
                Un_tilde = mode_dot(Un_tilde, Hermitian(Un), j)
            else:
                continue

        Un_f, _, _ = svd(tl.unfold(Un_tilde, i))

        U_list.append(Un_f)

    S = X
    for k, Un in enumerate(U_list):
        S = mode_dot(S, Hermitian(Un), k)
        
    return S, U_list

In [13]:
S, U_list = HOOI(X)

mode_dot(mode_dot(mode_dot(S, U_list[0], 0), U_list[1], 1), U_list[2], 2)

array([[[ 1.76405235+1.62434536j,  0.40015721-0.61175641j,
          0.97873798-0.52817175j,  2.2408932 -1.07296862j],
        [ 1.86755799+0.86540763j, -0.97727788-2.3015387j ,
          0.95008842+1.74481176j, -0.15135721-0.7612069j ],
        [-0.10321885+0.3190391j ,  0.4105985 -0.24937038j,
          0.14404357+1.46210794j,  1.45427351-2.06014071j]],

       [[ 0.76103773-0.3224172j ,  0.12167502-0.38405435j,
          0.44386323+1.13376944j,  0.33367433-1.09989127j],
        [ 1.49407907-0.17242821j, -0.20515826-0.87785842j,
          0.3130677 +0.04221375j, -0.85409574+0.58281521j],
        [-2.55298982-1.10061918j,  0.6536186 +1.14472371j,
          0.8644362 +0.90159072j, -0.74216502+0.50249434j]]])

In [14]:
X

array([[[ 1.76405235+1.62434536j,  0.40015721-0.61175641j,
          0.97873798-0.52817175j,  2.2408932 -1.07296862j],
        [ 1.86755799+0.86540763j, -0.97727788-2.3015387j ,
          0.95008842+1.74481176j, -0.15135721-0.7612069j ],
        [-0.10321885+0.3190391j ,  0.4105985 -0.24937038j,
          0.14404357+1.46210794j,  1.45427351-2.06014071j]],

       [[ 0.76103773-0.3224172j ,  0.12167502-0.38405435j,
          0.44386323+1.13376944j,  0.33367433-1.09989127j],
        [ 1.49407907-0.17242821j, -0.20515826-0.87785842j,
          0.3130677 +0.04221375j, -0.85409574+0.58281521j],
        [-2.55298982-1.10061918j,  0.6536186 +1.14472371j,
          0.8644362 +0.90159072j, -0.74216502+0.50249434j]]])